# Web Scraping with lazy load

Page to be scraped: AlianzaVerde

In [1]:
# Importing required libraries

import time

from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from pymongo import TEXT

from pymongo import MongoClient

In [2]:
# Creating a connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['candidatosSenado']
collection = db['candidatos']

In [3]:
# Base URL of the site to be analyzed
SITE_URL = 'https://alianzaverde.org.co'

In [4]:
# Firefox web driver path
# Download the driver for you S.O. here: https://github.com/mozilla/geckodriver/releases
DRIVER_PATH = './geckodriver-v0.31.0-win64/geckodriver.exe'

In [5]:
# Creating a new Edge window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

C:\Users\lsofi\AppData\Local\Temp\ipykernel_21276\851349788.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [331]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL + relative_path)
    prueba=0
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollBy(0, 700);')
        time.sleep(3)
        height = (browser.execute_script('return document.body.scrollHeight;'))
        if height == check_height: 
            break 
        height = check_height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [337]:

def cardsTotal(soup, cards):
    i=0
    # Finding the section where news are contained 
    layout = soup.find(class_ = 'main-body')
    # Getting blocks from layout
    blocks = layout.find_all(class_ = 'sppb-container-inner', recursive = True)
    bloksTotal=blocks
    while i<len(blocks):
        # Finding and concatenating news cards
        cards += blocks[i].find_all(class_ = 'sppb-person-image')
        i+=1
    return(cards)

In [338]:
# Getting HTML content for a particular news listing page
#Politica
totalBlocks=[]

#Se llena un arreglo con la informacion de cada card
candidatos = make_request(browser, '/lideres-verdes/congreso-2022/senado-2022')
candidatos

<html dir="ltr" lang="es-co" prefix="og: http://ogp.me/ns#"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<link href="https://alianzaverde.org.co/lideres-verdes/congreso-2022/senado-2022" rel="canonical"/>
<base href="https://alianzaverde.org.co/lideres-verdes/congreso-2022/senado-2022"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="Política, Transparencia, Verde, Mockus, Claudia, Medio Ambiente, Claudia López, Camilo Romero, Elecciones" name="keywords"/>
<meta content="Página oficial del Partido Alianza Verde de Colombia" name="description"/>
<meta content="HELIX_ULTIMATE_GENERATOR_TEXT" name="generator"/>
<title>Senado 2022</title>
<link href="/templates/ts_vinazine/images/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
<link href="https://alianzaverde.org.co/component/search/?Itemid=160&amp;format=opensearch" rel="search" title="Buscar Partido Alianza Ve

In [339]:
cardsTotal(candidatos, totalBlocks)

[<div class="sppb-person-image"><img alt="" class="sppb-img-responsive" loading="lazy" src="/images/Candidatos_2022/senado/2.jpg"/><div class="person-content-show-on-hover"><div class="person-content-hover-content-wrap"><span class="sppb-person-designation"><div style="text-align: justify;"><span style="font-size: 12pt;">Abogada y filósofa, con un máster en Ciencias de la Administración Pública, en Alemania. Experta en gestión pública, descentralización, transparencia, buen gobierno, modernización del estado, cultura ciudadana y cooperación internacional, temas en los cuales se ha desempeñado alternativamente como servidora pública, consultora y docente.</span></div></span><div class="sppb-person-social-icons"><ul class="sppb-person-social"><li><a aria-label="Facebook" href="https://twitter.com/VivianaBarbere " rel="noopener noreferrer" target="_blank"><i aria-hidden="true" class="fab fa-facebook-f" title="Facebook"></i></a></li><li><a aria-label="Twitter" href="https://twitter.com/Viv

In [380]:
print(len(totalBlocks))

34


In [387]:
# Building a list with title and relative path of the news founded
news = []

for card in totalBlocks: 
    try:

        news.append({
            'foto': card.find('img', class_ = 'sppb-img-responsive').get('src'),
            'descripcion': card.find('span', class_ = 'sppb-person-designation').get_text(),
            'redes': card.find('ul', class_ = 'sppb-person-social').find('a')['href']
        })

    except:
        news.append({
            'foto': card.find('img', class_ = 'sppb-img-responsive').get('src'),
            'descripcion': card.find('span', class_ = 'sppb-person-designation').get_text()
        })

        
        
   
     

In [388]:
news

[{'foto': '/images/Candidatos_2022/senado/2.jpg',
  'descripcion': 'Abogada y filósofa, con un máster en Ciencias de la Administración Pública, en Alemania. Experta en gestión pública, descentralización, transparencia, buen gobierno, modernización del estado, cultura ciudadana y cooperación internacional, temas en los cuales se ha desempeñado alternativamente como servidora pública, consultora y docente.',
  'redes': 'https://twitter.com/VivianaBarbere '},
 {'foto': '/images/Candidatos_2022/senado/3.jpg',
  'descripcion': 'Nacida de seres excepcionales, en tiempos inciertos; es la mujer de retos resueltos. Desde el inicio de su carrera política, su liderazgo se enfoca en mantener la cultura de la equidad, la familia y el fortalecimiento económico, político y social de las regiones en Colombia',
  'redes': 'https://www.facebook.com/sandraortizsenadora'},
 {'foto': '/images/Candidatos_2022/senado/5.jpg',
  'descripcion': 'Destacado Senador de la República en el año 2010 fue disciplinado 

In [357]:
# Storing extracted information for further analysis
collection.insert_many(news)

In [358]:
collection.create_index([('foto', TEXT),('descripcion', TEXT),('redes', TEXT)],default_language='spanish')

'foto_text_descripcion_text_redes_text'